In [1]:
import os
os.environ['AOC_SESSION'] = '53616c7465645f5fe53b184f367847aa55a2617e1b05248cac098b2c3dd14dae217169b3ce2e42b10bcc2af6d9b6787a4792a557f9705bb1eba86d761f2407d2'

In [2]:
import aocd
from aocd.models import Puzzle
day = 15
year = 2022
puzzle = Puzzle(year=year, day=day)
# data = aocd.get_data(day=day, year=year)
with open('./data/input_{:02d}'.format(day), 'w') as fh:
    fh.write(puzzle.input_data)

In [3]:
import re
import matplotlib.pyplot as plt

In [4]:
test_data = """Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3"""

In [5]:
data = puzzle.input_data.splitlines()
# data = test_data.splitlines()
len(data), data[:10]

(31,
 ['Sensor at x=3859432, y=2304903: closest beacon is at x=3677247, y=3140958',
  'Sensor at x=2488890, y=2695345: closest beacon is at x=1934788, y=2667279',
  'Sensor at x=3901948, y=701878: closest beacon is at x=4095477, y=368031',
  'Sensor at x=2422190, y=1775708: closest beacon is at x=1765036, y=2000000',
  'Sensor at x=2703846, y=3282799: closest beacon is at x=2121069, y=3230302',
  'Sensor at x=172003, y=2579074: closest beacon is at x=-77667, y=3197309',
  'Sensor at x=1813149, y=1311283: closest beacon is at x=1765036, y=2000000',
  'Sensor at x=1704453, y=2468117: closest beacon is at x=1934788, y=2667279',
  'Sensor at x=1927725, y=2976002: closest beacon is at x=1934788, y=2667279',
  'Sensor at x=3176646, y=1254463: closest beacon is at x=2946873, y=2167634'])

In [6]:
def dist(pos1, pos2):
    return int(abs(pos2.real - pos1.real) + abs(pos2.imag - pos1.imag))

def parse(data):
    sensors = []
    beacons = []
    for line in data:
        x1, y1, x2, y2 = re.findall('[-\d]+', line)
        sensors.append(int(x1) + int(y1)* 1j)
        beacons.append(int(x2)+ int(y2)*1j)
    return sensors, beacons

def intersect(row, sensor, dist):
    row_dist = abs(sensor.imag - row)
    if row_dist > dist:
        return []
    delta = dist - row_dist
    return [int(sensor.real - delta), int(sensor.real + delta)]

def merge_intervals(intervals):
    itvs = sorted(intervals)
    idx = 0
    for i in range(1, len(itvs)):
        if itvs[idx][1] >= itvs[i][0]:
            itvs[idx][1] = max(itvs[idx][1], itvs[i][1])
        else:
            idx += 1
            itvs[idx] = itvs[i]
    return itvs[:idx+1]

def get_nonvalid(row, sensors, dists):
    intersects = [intersect(row, sensor, dist)  for sensor, dist in zip(sensors, dists)] 
    intersects = sorted([i for i in intersects if i != []])
    # intervals = merge_intervals(intersects)
    return intersects

In [7]:
sensors, beacons = parse(data)

In [8]:
dists = [dist(p1, p2) for p1, p2 in zip(sensors, beacons)]


In [9]:
row = 2000000
non_valid = merge_intervals(get_nonvalid(row, sensors, dists))
res = sum([itv[1]-itv[0] for itv in non_valid])
res

4876693

In [154]:
puzzle.answer_a = res

That's the right answer!  You are one gold star closer to collecting enough star fruit. [Continue to Part Two]


In [10]:
non_valid

[[-303924, 4572769]]

# Part 2

In [11]:
%%time
for y in range(4000000):
    non_valid = merge_intervals(get_nonvalid(y, sensors, dists))
    if len(non_valid) > 1:
        print(y, non_valid)
        break

2855041 [[-419935, 2911362], [2911364, 4327534]]
CPU times: user 46.3 s, sys: 7.33 ms, total: 46.3 s
Wall time: 46.3 s


In [12]:
x = non_valid[0][1] + 1
tuning_frq = x * 4000000 + y
tuning_frq

11645454855041

In [162]:
puzzle.answer_b = tuning_frq

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 15! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


In [26]:
def search_quadrant(x1, y1, x2, y2, sensors_dists):
    quadrants = []
    # print(x1, y1, x2, y2)
    if (abs(x2 - x1) < 100) or (abs(y2 - y1) < 100):
        # print (x1, y1, x2, y2)
        return [(x1, y1, x2, y2)]
    
    corners = [x1+y1*1j, x1+y2*1j, x2+y1*1j, x2+y2*1j]
    cnt = True
    for sensor, dst in sensors_dists:
        if all([dist(corner, sensor) <= dst for corner in corners]):
            # print('not continuing')
            cnt = False
            return []
    if cnt:
        w = abs(x2 - x1) // 2
        h = abs(y2 - y1) // 2
        quadrants += search_quadrant(x1    , y1    , x1+w, y1+h, sensors_dists)
        quadrants += search_quadrant(x1+w+1, y1    , x2  , y1+h, sensors_dists)
        quadrants += search_quadrant(x1    , y1+h+1, x1+w, y2  , sensors_dists)
        quadrants += search_quadrant(x1+w+1, y1+h+1, x2  , y2  , sensors_dists)
        
    return quadrants

def check_point(point, sensors_dists):
    for sensor, dst in sensors_dists:
        if dist(sensor, point) <= dst:
            return False, dst - dist(sensor, point)
    return True, 0

def check_sensors(sensors_dists, maxpos=4000000):
    for sensor, dist in sensors_dists:
        print(f"checking sensor {sensor} w/ range {dist}")
        pos = sensor - 1j*(dist+1)
        step = 1 + 1j
        while (pos.real <= maxpos) and (pos.imag <= maxpos) and (pos.imag >= 0) and (pos.real <= (sensor.real+dist)):
            fnd, dst = check_point(pos, [(sns, dst) for sns, dst in sensors_dists if sns != sensor])
            if fnd:
                return pos
            pos += (step * max(dst//2, 1))
            # print(pos)
        # break   
  

In [28]:
%%time
free = check_sensors(list(zip(sensors, dists)))
free

checking sensor (3859432+2304903j) w/ range 1018240
checking sensor (2488890+2695345j) w/ range 582168
checking sensor (3901948+701878j) w/ range 527376
checking sensor (2422190+1775708j) w/ range 881446
checking sensor (2703846+3282799j) w/ range 635274
CPU times: user 661 µs, sys: 0 ns, total: 661 µs
Wall time: 503 µs


(2911363+2855041j)

In [30]:
x = free.real
y = free.imag
tuning_frq = x * 4000000 + y
int(tuning_frq)

11645454855041